Text Summarizing

In [85]:
from summarizer import Summarizer
from transformers import logging, AutoConfig, AutoTokenizer, AutoModel
logging.set_verbosity_error()
import warnings
warnings.filterwarnings("ignore")

# alternate: bert-large-uncased (better summary)
custom_config = AutoConfig.from_pretrained('mrm8488/bert-tiny-finetuned-squadv2')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('mrm8488/bert-tiny-finetuned-squadv2')
custom_model = AutoModel.from_pretrained('mrm8488/bert-tiny-finetuned-squadv2', config=custom_config)

model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

In [87]:
f = open("sample.txt","r")
full_text = f.read()
result = model(full_text, min_length=60, max_length = 500 , ratio = 0.4)
summarized_text = ''.join(result)
print (summarized_text)

India has a very ancient tradition of art, which has exchanged many influences with the rest of Eurasia, especially in the first millennium, when Buddhist art spread with Indian religions to Central, East and South-East Asia, the last also greatly influenced by Hindu art.[377] Thousands of seals from the Indus Valley Civilization of the third millennium BCE have been found, usually carved with animals, but a few with human figures. The "Pashupati" seal, excavated in Mohenjo-daro, Pakistan, in 1928–29, is the best known.[378][379] After this there is a long period with virtually nothing surviving.[379][380] Almost all surviving ancient Indian art thereafter is in various forms of religious sculpture in durable materials, or coins.


In [72]:
full_text

"here are the letters the Romans gave us the near the countries whose languages derived from Latin today they cover half the world as for the ancient Romans the boundaries of the states encompassed their entire civilization the Roman peace or Pax Romana serves as the first example of globalization let's take a walk across 12 centuries of the Roman history and yes those numerals are also a Roman legacy what is Rome City on seven hills capital of Italy but that is today two thousand years ago there it is another thousand years ago they're a tiny tribal settlement of the Latins by the river Tiber how did this manage to conquer the world first it was lucky with its neighbors to the north the Etruscans of modern Tuscany and mysterious people whose language has never been fully decipher to the south Greek colonies these peoples all traded with each other it was at the crossroads of their trade routes that Rome appeared from the very start bromas villain open city a safe haven for outcasts mu

extract N (best) nouns from text

In [127]:
import nltk
nltk.download('stopwords')

import pke
import string
from nltk.corpus import stopwords

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [91]:
def get_n_nouns(text, n):
  text = text.lower()
  
  extractor = pke.unsupervised.MultipartiteRank()
  stoplist = list(string.punctuation)
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  extractor.load_document(input=text, stoplist=stoplist)

  pos = {'PROPN'}

  extractor.candidate_selection(pos=pos)
  max_n = len(extractor.candidates)

  if (len(extractor.candidates) < max_n):
    return [i for i in extractor.candidates if i in text]

  for i in range(1, max_n+1):
    extractor.candidate_weighting(alpha=1.1,
                                    threshold=0.75,
                                    method='average')
    nouns = [i[0] for i in extractor.get_n_best(i) if i[0] in text]
    if len(nouns) == n:
      return nouns

  return nouns

In [92]:
filtered_keys = get_n_nouns(full_text, 4)

In [119]:

nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if 20 < len(sentence) < 300]
    return sentences
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)
    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences
sentences = tokenize_sentences(summarized_text)
keyword_sentence_mapping = get_sentences_for_keyword(get_n_nouns(full_text, 4), sentences)    
print (keyword_sentence_mapping)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chiragmanjeshwar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'hindu': ['India has a very ancient tradition of art, which has exchanged many influences with the rest of Eurasia, especially in the first millennium, when Buddhist art spread with Indian religions to Central, East and South-East Asia, the last also greatly influenced by Hindu art.'], 'ellora': [], 'karla': [], 'east': ['India has a very ancient tradition of art, which has exchanged many influences with the rest of Eurasia, especially in the first millennium, when Buddhist art spread with Indian religions to Central, East and South-East Asia, the last also greatly influenced by Hindu art.', 'India has a very ancient tradition of art, which has exchanged many influences with the rest of Eurasia, especially in the first millennium, when Buddhist art spread with Indian religions to Central, East and South-East Asia, the last also greatly influenced by Hindu art.']}


In [124]:
def get_max_n(text):
  extractor = pke.unsupervised.MultipartiteRank()
  stoplist = list(string.punctuation)
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  extractor.load_document(input=text, stoplist=stoplist)

  pos = {'PROPN'}

  extractor.candidate_selection(pos=pos)
  max_n = len(extractor.candidates)

  return max_n

import functools
def get_size(keyword_sentence_mapping):
  return functools.reduce(lambda x, y: x + len(y), keyword_sentence_mapping.values(), 0)

def get_n_sentence_mapping(text, n):

  max_n = get_max_n(text)
  for i in range(1, max_n):
    sentences = tokenize_sentences(summarized_text)
    keyword_sentence_mapping = get_sentences_for_keyword(get_n_nouns(full_text, i), sentences)
    if get_size(keyword_sentence_mapping) == n:
      break
  
  return keyword_sentence_mapping

  


In [125]:
get_size(keyword_sentence_mapping)

3

In [120]:
keyword_sentence_mapping

{'hindu': ['India has a very ancient tradition of art, which has exchanged many influences with the rest of Eurasia, especially in the first millennium, when Buddhist art spread with Indian religions to Central, East and South-East Asia, the last also greatly influenced by Hindu art.'],
 'ellora': [],
 'karla': [],
 'east': ['India has a very ancient tradition of art, which has exchanged many influences with the rest of Eurasia, especially in the first millennium, when Buddhist art spread with Indian religions to Central, East and South-East Asia, the last also greatly influenced by Hindu art.',
  'India has a very ancient tradition of art, which has exchanged many influences with the rest of Eurasia, especially in the first millennium, when Buddhist art spread with Indian religions to Central, East and South-East Asia, the last also greatly influenced by Hindu art.']}

In [126]:
import requests
import re
import random
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn

nltk.download('averaged_perceptron_tagger')

# Distractors from Wordnet
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

def get_wordsense(sent,word):
    word= word.lower()
    
    if len(word.split())>0:
        word = word.replace(" ","_")
    
    
    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

# Distractors from http://conceptnet.io/
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = [] 
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term'] 

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)
                   
    return distractor_list

key_distractor_list = {}

for keyword in keyword_sentence_mapping:
    wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense,keyword)
        if len(distractors) ==0:
            distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
    else:
        
        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors

index = 1
print ("#############################################################################")
print ("NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. ")
print ("#############################################################################\n\n")
for each in key_distractor_list:
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub( " _______ ", sentence)
    print ("%s)"%(index),output)
    choices = [each.capitalize()] + key_distractor_list[each]
    top4choices = choices[:4]
    random.shuffle(top4choices)
    optionchoices = ['a','b','c','d']
    for idx,choice in enumerate(top4choices):
        print ("\t",optionchoices[idx],")"," ",choice)
    print ("\nMore options: ", choices[4:20],"\n\n")
    index = index + 1
    

Warming up PyWSD (takes ~10 secs)... 

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - '/Users/chiragmanjeshwar/nltk_data'
    - '/opt/homebrew/opt/python@3.11/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/opt/homebrew/opt/python@3.11/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/opt/homebrew/opt/python@3.11/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [116]:
full_text

'India has a very ancient tradition of art, which has exchanged many influences with the rest of Eurasia, especially in the first millennium, when Buddhist art spread with Indian religions to Central, East and South-East Asia, the last also greatly influenced by Hindu art.[377] Thousands of seals from the Indus Valley Civilization of the third millennium BCE have been found, usually carved with animals, but a few with human figures. The "Pashupati" seal, excavated in Mohenjo-daro, Pakistan, in 1928–29, is the best known.[378][379] After this there is a long period with virtually nothing surviving.[379][380] Almost all surviving ancient Indian art thereafter is in various forms of religious sculpture in durable materials, or coins. There was probably originally far more in wood, which is lost. In north India Mauryan art is the first imperial movement.[381][382][383] In the first millennium CE, Buddhist art spread with Indian religions to Central, East and South-East Asia, the last also 

In [84]:
summarized_text

''